In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/capstone')
print(os.getcwd())

/content
/content/drive/MyDrive/capstone


- 필요한 함수 구현

In [ ]:
def save_video(out_video_name):
  input_video_path = out_video_name
  # MP4 동영상 저장 경로
  output_video_path = input_video_path[:-3]+"mp4"

  # 동영상 파일 열기
  cap = cv2.VideoCapture(input_video_path)

  # 동영상 저장을 위한 VideoWriter 설정
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 형식으로 저장
  out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

  # 동영상 파일에서 프레임을 읽어 MP4 동영상 파일로 저장
  while True:
      ret, frame = cap.read()
      if not ret:
          break
      out.write(frame)

  # 자원 해제
  cap.release()
  out.release()

- 개발 환경 설정

In [ ]:
# 필요한 라이브러리 임포트
import cv2
import numpy as np

In [ ]:
# YOLO 설정 및 가중치 파일 경로

model = "yolov3"  # yolov3, yolov3-tiny

if model == "yolov3":
  config_path = '/content/drive/MyDrive/capstone/yolov3.cfg'
  weights_path = '/content/drive/MyDrive/capstone/yolov3.weights'
  labels_path = '/content/drive/MyDrive/capstone/coco.names'
elif model == "yolov3-tiny":
  config_path = '/content/drive/MyDrive/capstone/yolov3-tiny.cfg'
  weights_path = '/content/drive/MyDrive/capstone/yolov3-tiny.weights'
  labels_path = '/content/drive/MyDrive/capstone/coco.names'

# 레이블 및 색상 로드
with open(labels_path, 'r') as f:
    labels = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

# YOLO 네트워크 로드
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layer_indexes = net.getUnconnectedOutLayers().flatten() - 1
output_layers = [layer_names[i] for i in output_layer_indexes]

In [ ]:
# 동영상 열기
video_file_name = "/content/drive/MyDrive/capstone/dataset/data.mov"
out_video_name = "/content/drive/MyDrive/capstone/dataset/result.avi"
cap = cv2.VideoCapture(video_file_name)

# 동영상 저장을 위한 VideoWriter 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_video = cv2.VideoWriter(out_video_name, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
# out_video = cv2.VideoWriter(out_video_name, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

In [ ]:
# Get video dimensions
width = int(cap.get(3))
height = int(cap.get(4))

# Define multiple ROIs
# data 3,4
rois = [
    {"x": 65, "y": 190, "width": 110, "height": 130},
    {"x": 250, "y": 130, "width": 115, "height": 140},
    {"x": 395, "y": 80, "width": 95, "height": 110},
    {"x": 510, "y": 85, "width": 70, "height": 85},
		{"x": 615, "y": 90, "width": 50, "height": 61}
]


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

# video_file_name = "/content/drive/MyDrive/capstone/data.mov"
cap = cv2.VideoCapture(video_file_name)
ret, frame = cap.read()
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for displaying in matplotlib

roi = [0, 0, 200, 200]  # Initial ROI [x, y, width, height]

def update(x=0, y=0, width=200, height=200):
    roi[0], roi[1], roi[2], roi[3] = x, y, width, height
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.imshow(frame)
    rect = plt.Rectangle((x, y), width, height, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    plt.show()

interact(update, x=(0, frame.shape[1]-1), y=(0, frame.shape[0]-1), width=(1, frame.shape[1]), height=(1, frame.shape[0]));

# 75 270 110 130
# 270 220 95 110
# 420 210 60 80
# 510 200 45 56
# 580 190 40 52

interactive(children=(IntSlider(value=0, description='x', max=853), IntSlider(value=0, description='y', max=55…

## human detection

In [ ]:
empty_time = {i: 0 for i in range(len(rois))}

# Set a flag to check if a person is inside an ROI
person_inside = {i: False for i in range(len(rois))}

frame_count = 0
boxes = []
confidences = []
class_ids = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Reset the detection flags for all ROIs
    detections_in_roi = [False] * len(rois)

    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    for out in outs:
        for detection in out:
            # ... [객체 검출 관련 코드] ...
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                if labels[class_id] == 'person':
                    # ... [사람 바운딩 박스 그리기 코드] ...
                    center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                    x, y = int(center_x - w / 2), int(center_y - h / 2)
                    boxes.append((x, y, w, h))  # Add the person bounding box to the list
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.4)  # NMS 적용

    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(labels[class_ids[i]])
        color = (0, 127, 255)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        for i, roi in enumerate(rois):
            if (x < (roi["x"] + roi["width"]) and x + w > roi["x"] and
              y < (roi["y"] + roi["height"]) and y + h > roi["y"]):
                detections_in_roi[i] = True
                empty_time[i] = 0  # Reset the timer if a person is detected

    # Draw rectangles for each ROI
    for i, roi in enumerate(rois):
        if detections_in_roi[i]:
            color = (0, 255, 0)  # Red for ROIs with detections
        else:
            color = (0, 0, 255)  # Green for empty ROIs
            empty_time[i] += 1  # Increase the timer if no person is detected

        cv2.rectangle(frame, (roi["x"], roi["y"]), (roi["x"] + roi["width"], roi["y"] + roi["height"]), color, 2)

        # Show the empty time counter on the video
        cv2.putText(frame, f"Empty: {empty_seconds:.1f} s", (roi["x"], roi["y"] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


    out_video.write(frame)

    boxes.clear()        # 초기화
    confidences.clear()  # 초기화
    class_ids.clear()    # 초기화

# 자원 해제
cap.release()
out_video.release()

save_video(out_video_name)